In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.tools import tool
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.agent import AgentFinish
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from pydantic import BaseModel, Field
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from pydantic import BaseModel, Field

In [2]:
openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")

C:\Users\Design Dep\AppData\Local\Temp\ipykernel_8616\1305393998.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  openai_llm = ChatOpenAI(temperature=0, model="gpt-4o")


In [3]:
chat_template ="""

You are an expert in analyzing CVs and providing personalized career advice. Given an individual's CV, you will:

1. Conduct a detailed analysis of their skills, experience, qualifications, and overall suitability for various roles.
2. Recommend the best job opportunities that align with their profile, ensuring that the suggestions are well-matched to their strengths and career goals.
3. Take into account current industry trends, job market demand, and emerging opportunities to ensure that your recommendations are both relevant and forward-thinking.
4. Provide actionable insights that can help improve the CV, such as highlighting areas for growth or skill enhancement based on market needs.

Your goal is to give comprehensive, data-driven advice that helps individuals make informed career decisions.

below is the CV:
{input}
"""



In [4]:
import gradio as gr
import os
def get_file(file_path):
    directory_name = os.path.dirname(file_path)  
    
    documents = SimpleDirectoryReader(directory_name).load_data()    
    prompt = ChatPromptTemplate.from_template(chat_template)
    chain = LLMChain(prompt=prompt, llm=openai_llm)
    global doc_info 
    doc_info = documents[0].text
    response = chain.run({"input":doc_info})
    return response
demo = gr.Interface(fn=get_file, inputs=gr.File(), outputs="text")




d:\Rhodrick\langChain\Job application assistant\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
demo.launch()
# demo.close()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


C:\Users\Design Dep\AppData\Local\Temp\ipykernel_8616\1428651765.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(prompt=prompt, llm=openai_llm)
C:\Users\Design Dep\AppData\Local\Temp\ipykernel_8616\1428651765.py:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run({"input":doc_info})


In [ ]:
class CVDetails(BaseModel):
    cv_summary :str = Field(description="The overall summary of the person Cv")

In [ ]:
from bs4 import BeautifulSoup
import requests

html_text = requests.get("https://www.myjob.mu/Jobs/ICT-IT-Web/").text

soup = BeautifulSoup(html_text, "lxml")
job_results = soup.find_all("div", class_="module job-result")
if len(job_results)> 0:
    for job in job_results:
        company_name = job.find('h3', itemprop="name")
        company_name = company_name.a.text
        print(f"company_name is {company_name}")
        job_name = job.find("h2", itemprop="title")
        job_name = job_name.a.text
        job_results_overview = job.find("div", class_="job-result-overview")
        overview = job_results_overview.find("")
        print(f"job name is: {job_name}")
        #job_description =

